All features are extracted over 3 second segmented of the audio clip(1 song = 10 parts of 3 seconds each)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import librosa
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import csv



In [ ]:
song_dir= '/content/drive/My Drive/RP/genres/'

In [ ]:
def get_features(y, sr):
  features = {'centroid': None, 'rolloff': None, 'flux': None, 'rmse': None,
                'zcr': None, 'contrast': None, 'bandwidth': None, 'flatness': None, 'chroma_stft': None}
    
  # Using librosa to calculate features
  features['chroma_stft']=librosa.feature.chroma_stft(y=y, sr=sr).ravel()
  features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr).ravel()
  features['rolloff'] = librosa.feature.spectral_rolloff(y, sr=sr,).ravel()
  features['zcr'] = librosa.feature.zero_crossing_rate(y).ravel()
  features['rmse'] = librosa.feature.rms(y).ravel()
  features['flux'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
  features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr).ravel()
  features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr).ravel()
  features['flatness'] = librosa.feature.spectral_flatness(y).ravel()
    
  #  Using librosa to calculate MFCC 
  mfcc = librosa.feature.mfcc(y, sr=sr, n_mfcc=13)
  for idx, v_mfcc in enumerate(mfcc):
    features['mfcc_{}'.format(idx)] = v_mfcc.ravel()
        
  # calculating statistical moments for all features
  def get_moments(descriptors):
    result = {}
    for k, v in descriptors.items():
      result['{}_max'.format(k)] = np.max(v)
      result['{}_min'.format(k)] = np.min(v)
      result['{}_mean'.format(k)] = np.mean(v)
      result['{}_std'.format(k)] = np.std(v)
      result['{}_kurtosis'.format(k)] = kurtosis(v)
      result['{}_skew'.format(k)] = skew(v)
    return result
    
  dict_agg_features = get_moments(features)
    
  return dict_agg_features

In [ ]:
num_segments=10 #change acc to requirement eg: 5, 10
sample_rate= 22050
track_duration = 30 
samples_per_track = sample_rate * track_duration #22050*30 = 661500
samples_per_segment = int(samples_per_track / num_segments) # 66150/5 = 132300

#navigating through song dir -> genre folder -> song
def final_extraction(song_dir):
  genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
  last_features=[]
  for genre in genres:
    path_genre = song_dir + '/' + genre  
    for song in os.listdir(path_genre):        
      path_song = path_genre + '/' + song
      y, sr = librosa.load(path_song,sr=sample_rate)
      for d in range(num_segments): #break each 30 sec audio into 5 segments and extract features from all
        # calculate start and finish sample for current segment
        start = samples_per_segment * d #d=0, start=0
        finish = start + samples_per_segment # 0 + 132300
        #print("start:",start)
        #print("finish",finish)
        features = get_features(y[start:finish], sr)
        features['genre'] = genre
        last_features.append(features)
        
  return pd.DataFrame(last_features)

In [ ]:
%%time

df_features = final_extraction(song_dir)
df_features.to_csv('Data.csv', index=False)


CPU times: user 18min 11s, sys: 14min 58s, total: 33min 10s
Wall time: 16min 55s


In [ ]:
df_features = pd.read_csv('Data.csv')

In [ ]:
df_features.shape

(10000, 133)

In [ ]:
df_features.head()

,centroid_max,centroid_min,centroid_mean,centroid_std,centroid_kurtosis,centroid_skew,rolloff_max,rolloff_min,rolloff_mean,rolloff_std,rolloff_kurtosis,rolloff_skew,flux_max,flux_min,flux_mean,flux_std,flux_kurtosis,flux_skew,rmse_max,rmse_min,rmse_mean,rmse_std,rmse_kurtosis,rmse_skew,zcr_max,zcr_min,zcr_mean,zcr_std,zcr_kurtosis,zcr_skew,contrast_max,contrast_min,contrast_mean,contrast_std,contrast_kurtosis,contrast_skew,bandwidth_max,bandwidth_min,bandwidth_mean,bandwidth_std,...,mfcc_6_std,mfcc_6_kurtosis,mfcc_6_skew,mfcc_7_max,mfcc_7_min,mfcc_7_mean,mfcc_7_std,mfcc_7_kurtosis,mfcc_7_skew,mfcc_8_max,mfcc_8_min,mfcc_8_mean,mfcc_8_std,mfcc_8_kurtosis,mfcc_8_skew,mfcc_9_max,mfcc_9_min,mfcc_9_mean,mfcc_9_std,mfcc_9_kurtosis,mfcc_9_skew,mfcc_10_max,mfcc_10_min,mfcc_10_mean,mfcc_10_std,mfcc_10_kurtosis,mfcc_10_skew,mfcc_11_max,mfcc_11_min,mfcc_11_mean,mfcc_11_std,mfcc_11_kurtosis,mfcc_11_skew,mfcc_12_max,mfcc_12_min,mfcc_12_mean,mfcc_12_std,mfcc_12_kurtosis,mfcc_12_skew,genre
0,2968.610553,849.658283,1544.599741,375.803588,1.831593,0.885106,5124.902344,1152.026367,2923.463604,885.025317,-0.580839,0.136526,8.759687,0.0,1.673339,2.042800,2.179828,1.733197,0.133169,0.021886,0.080475,0.025623,-0.657055,-0.079778,0.190918,0.036133,0.086009,0.028843,2.138670,1.110720,45.732313,8.504636,21.857460,6.934282,0.460795,0.903689,1988.241864,1186.624526,1578.929816,198.308260,...,7.343157,0.279836,-0.245533,16.085646,-21.092693,-0.045477,7.397243,-0.263076,-0.234852,4.270060,-32.617149,-14.467911,7.470672,-0.471515,0.129063,35.907036,-11.864138,3.204973,8.351996,2.065392,1.171183,19.338140,-32.714924,-19.285934,8.435503,7.219372,2.304931,28.912182,-12.308210,2.815688,8.808426,-0.260792,0.575162,12.387854,-28.359610,-10.360458,10.190949,-0.724335,0.354533,blues
1,4429.203787,907.213204,2388.580024,905.798802,-0.714752,0.241953,6890.625000,1367.358398,4544.582520,1599.046449,-0.827402,-0.498110,8.783724,0.0,1.362969,1.329510,9.201735,2.577901,0.288371,0.013944,0.112698,0.069025,-0.434302,0.681183,0.365723,0.033691,0.147994,0.080016,-0.294200,0.735440,54.700207,7.934002,22.485827,9.107351,1.708504,1.423087,2555.903287,1244.600989,1967.856374,346.393644,...,13.217931,-0.062808,-0.812195,25.972910,-23.918947,7.619702,10.936044,-0.794936,-0.228306,21.045078,-43.906151,-18.526569,15.031443,0.118298,0.817024,30.541256,-19.761093,6.837147,11.492730,-0.580217,-0.381661,0.232399,-36.592163,-17.526182,8.070435,-0.502971,-0.172180,24.164585,-25.312050,-5.920010,8.267508,0.725408,0.270310,19.701296,-21.291504,-3.452553,10.564160,-0.824106,0.241367,blues
2,3920.272603,1033.019916,2097.306892,641.573280,-0.537890,0.391426,6793.725586,1604.223633,4234.007474,1380.422189,-1.120051,-0.069798,12.446646,0.0,1.622100,2.203782,7.413245,2.599538,0.228752,0.010686,0.097500,0.060494,-0.820085,0.524261,0.270508,0.030762,0.107741,0.046625,1.098290,1.133656,47.703467,7.770299,21.436428,8.704255,1.395805,1.407812,2657.946803,1342.833693,1965.804096,299.918228,...,10.595219,2.919454,-1.232423,31.932796,-9.238714,11.541316,10.228969,-1.151476,-0.025551,10.531494,-43.282532,-19.102686,13.371345,-0.935848,0.282861,22.897808,-7.069962,6.450355,8.402604,-1.137287,0.101380,-2.642356,-35.036060,-18.482765,7.005689,-0.535371,-0.211210,18.304348,-17.814165,-1.694399,6.966511,-0.179246,0.425301,16.281624,-26.752865,-6.195611,8.952841,-0.399917,0.148912,blues
3,3041.000699,1072.820372,1990.354892,518.423250,-1.102732,-0.114362,5921.630859,1722.656250,3912.665828,1143.723499,-1.013715,-0.411620,10.960407,0.0,1.704432,2.023815,4.507105,2.091851,0.197585,0.039848,0.102549,0.039995,-0.481531,0.561519,0.250000,0.057129,0.116095,0.041049,1.555917,1.277675,47.919475,5.914184,21.976961,8.148538,0.874961,1.120064,2318.260773,1169.677329,1767.231727,296.499584,...,8.654659,-0.426573,-0.394457,35.603943,-14.904453,5.250053,11.874259,-0.306106,0.715131,17.805515,-44.041222,-13.371712,13.097857,0.051051,-0.004489,30.755449,-8.647285,8.999457,10.627526,-1.143588,0.059382,14.695620,-39.088749,-16.1